<a href="https://colab.research.google.com/github/seraffin/FailOmen/blob/master/Mariusz/2monthEvaluationDecisionTree2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.ensemble import ExtraTreesRegressor

# Helper libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

from google.colab import drive

Using TensorFlow backend.


# Utilities

In [0]:
# Evaluation function

def evaluation(additionalPredictions, predictions, refYsupervisor, verbose = False):
  
  lenght = len(refYsupervisor)

  failPositions = [[] for y in range(lenght)]

  for i, a in enumerate(refYsupervisor):

    for j, b in enumerate(a):
      if b == 0 : failPositions[i].append(j);

  predictionsTemp = predictions.copy()
  predictionPositions = [[] for y in range(lenght)]


  for i, a in enumerate(predictionsTemp):

    if len(failPositions[i]) != 0:
      for j in range(1 + additionalPredictions):
        argmin = np.argmin(a)
        predictionPositions[i].append(argmin)
        predictionsTemp[i][argmin] = 1

  predictionHits = [[] for y in range(lenght)]

  for i, a in enumerate(failPositions):
    count = 0
    for j, b in enumerate(a):


      for c in predictionPositions[i]:
        if c == b : count += 1

    if len(failPositions) != 0:
      predictionHits[i] = count


  failsCount = 0
  hitsCount = 0
  for i, a in enumerate(refYsupervisor):
    j = 0

    for k, b in enumerate(a):
      if b < 1.0 : j += 1

  #  if j > 0 :
  #    print (i, '.', j, predictionHits[i])


    failsCount += j
    hitsCount += predictionHits[i]

  print('additional tests - ', "%.2f" % (hitsCount / failsCount * 100), '%')

  if verbose == True:
    for i in range(lenght):
      print(i ,len(failPositions[i]), len(predictionPositions[i]), predictionHits[i], sep='\t')

In [0]:
#Distribution of argmins through all the predictions
  
def argminsDistribution():
  
    i = 1
    tab = [0] * 542
    for a in predictions:
      j = 0
      for b in a:
        if b < 1.0 : j = j + 1

      #print (i, '. ', j, np.argmin(a))
      tab[np.argmin(a)] += 1
      i = i + 1

    i = 0
    for a in tab:
      if a > 0 : print ('position', i, '\targmin count', a)
      i += 1

In [0]:
def printFailDistributionStats(y):
  
  failCount = 0
  passCount = 0
  failBuildCount = 0

  for a in y:
    if a[np.argmin(a)] == 0 : failBuildCount += 1
    for b in a:
      if b == 0 : failCount += 1
      if b == 1 : passCount += 1
  print ('failBuildCount', failBuildCount)
  print ('passBuildCount', len(y) - failBuildCount)
  print ('failBuild share ', failBuildCount / len(y) * 100, '%', sep='')
  print ('failCount', failCount)
  print ('passCount', passCount)
  print ('fail share ', failCount / (failCount + passCount) * 100, '%', sep='')

In [0]:
#Distributions of argmins through all the predictions

def countArgminsDistribution(prediction):
  
  i = 1
  tab = [0] * 542
  for a in predictions:
    j = 0
    for b in a:
      if b < 1.0 : j = j + 1

    #print (i, '. ', j, np.argmin(a))
    tab[np.argmin(a)] += 1
    i = i + 1

  i = 0
  distributed_array = []
  for a in tab:
    if a > 0 : 
      print ('position', i, '\targmin count', a)
      distributed_array.append((i,a))
    i += 1
  create_plot(distributed_array)  
  return distributed_array

In [0]:
# Create a list of single member classes positions

def listSingleMemberClassesPositions(refY, printFlag):

  yList = refY.tolist()
  yDistinct = [ele for ind, ele in enumerate(yList) if ele not in yList[:ind]]

  classesPopulation = []
  singleMemberClassesPositions = []
  noSingleMemberClasses = 0

  for i, a in enumerate(yDistinct):
    population = 0
    memberPosition = 0
    for j, b in enumerate(yList):
      if a == b: 
        population += 1
        memberPosition = j

    classesPopulation.append(population)
  #  print(i, population)
    if population == 1: 
      noSingleMemberClasses += 1
      singleMemberClassesPositions.append(memberPosition)

  if printFlag == True: 
    print(noSingleMemberClasses)
  else:
    return singleMemberClassesPositions

In [0]:
def searchForFailed(yList, withpassed_data = False, share = 1):
  changedRowList = []
  not_add_passed_data = 0
  for row in range (len(yList)):
    if 0 in yList[row]:
      changedRowList.append(row)
      continue
    elif withpassed_data == True:
      if not not_add_passed_data % share:
        changedRowList.append(row)
    not_add_passed_data += 1
      
  return changedRowList

def returnFailedData(xList, yList, changedRowList):
  xFailed = []
  yFailed = []
  
  for row in changedRowList:
    xFailed.append(xList[row])
    yFailed.append(yList[row])
  xFailed = np.array(xFailed)
  yFailed = np.array(yFailed)
  return xFailed, yFailed

def searchForPassed(yList):
  changedRowList = []
  for row in range(len(yList)):
    if not 0 in yList[row]:
      changedRowList.append(row)

  return changedRowList

def searchForUnchangedFiles(refDatasetFileNumpy, refNoTestSamples):
  columnsSum = np.sum(refDatasetFileNumpy, axis=1)
  columnsToDelete = []
  for i, column in enumerate(columnsSum):
    if column == 0: columnsToDelete.append(i + refNoTestSamples)
  
  for element in columnsToDelete:
    print(element, columnsSum[element])
  return columnsToDelete

In [0]:
# Double the single member classes for stratified sampling

def appendToDataset(refSingleMemberClassesPositions, refDataset):
  
  newDataset = pd.DataFrame(columns=list(refDataset))
  for i, a in enumerate(refSingleMemberClassesPositions):
    newDataset = newDataset.append(refDataset.iloc[a, :])
  
  return refDataset.append(newDataset, ignore_index=True)

In [0]:
def checkGPUpresence():

  device_name = tf.test.gpu_device_name()
  if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
  print('Found GPU at: {}'.format(device_name))

# Prepare data

In [0]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#TODO : load data first check no columns, than add headers

#Importing dataset
noColumnsInFile = 22952
noTestsInFile = 402
fileColumnsEnd = noColumnsInFile - noTestsInFile
headers = list(range(0, noColumnsInFile))
dataset = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataToML_bin_20190117_new2.csv', names = headers)

In [0]:
noTestSamples = 200
currnetNoColumns = noColumnsInFile
currentFileColumsEnd = currnetNoColumns - noTestsInFile

In [0]:
datasetTestNumpy = dataset.iloc[noTestSamples:, fileColumnsEnd:noColumnsInFile].values
rowsToDelete = searchForPassed(datasetTestNumpy)
datasetFileNumpy = dataset.iloc[noTestSamples:, 0:fileColumnsEnd].values
columnsToDelete = searchForUnchangedFiles(datasetFileNumpy, noTestSamples)

dataset = dataset.drop(rowsToDelete, axis=0)
dataset = dataset.drop(columnsToDelete, axis=1)

currnetNoColumns = noColumnsInFile - len(columnsToDelete)
currentFileColumsEnd = currnetNoColumns - noTestsInFile

dataset.shape

(1736, 21136)

In [0]:
# Realistic split data set

x = dataset.iloc[noTestSamples:, 0:currentFileColumsEnd].values
xTest = dataset.iloc[:noTestSamples, 0:currentFileColumsEnd].values
y = dataset.iloc[noTestSamples:, currentFileColumsEnd:currnetNoColumns].values
yTest = dataset.iloc[:noTestSamples, currentFileColumsEnd:currnetNoColumns].values

NameError: ignored

In [0]:
# Stratified sampling split
x = dataset.iloc[:, 0:fileColumnsEnd].values
y = dataset.iloc[:, fileColumnsEnd:noColumnsInFile].values

In [0]:
printFailDistributionStats(y)

failBuildCount 2002
passBuildCount 0
failBuild share 100.0%
failCount 3679
passCount 801125
fail share 0.457129934741875%


In [0]:
printFailDistributionStats(yTest)

NameError: ignored

In [0]:
# Reduce overpopulated class with fully passed builds

xReduced, yReduced = returnFailedData(x, y, searchForFailed(y, withpassed_data=False, share = 1))
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(xReduced, yReduced, test_size = 0.2, random_state=77)

# Prepare and train model

In [0]:
xTrain.shape

(1601, 22550)

In [0]:
start_time = time.time()

model = Sequential()
model.add(Dense(15175, activation="sigmoid", input_dim=fileColumnsEnd, kernel_initializer="uniform"))
model.add(Dense(15175, activation="sigmoid", kernel_initializer="uniform"))
model.add(Dense(noTestsInFile, activation="sigmoid", kernel_initializer="uniform"))

model.compile(optimizer = tf.train.MomentumOptimizer(learning_rate = 0.01, momentum = 0.4), loss = 'binary_crossentropy', metrics = ['accuracy'])

model.fit(xTrain, yTrain, batch_size = 20, epochs = 15)

print("--- %s seconds ---" % (time.time() - start_time))

Epoch 1/15
1601/1601 [==============================] - 26s 16ms/step - loss: 0.2208 - acc: 0.9340
Epoch 2/15
1601/1601 [==============================] - 16s 10ms/step - loss: 0.0478 - acc: 0.9956
Epoch 3/15
1601/1601 [==============================] - 16s 10ms/step - loss: 0.0370 - acc: 0.9956
Epoch 4/15
1601/1601 [==============================] - 16s 10ms/step - loss: 0.0329 - acc: 0.9956
Epoch 5/15
1601/1601 [==============================] - 16s 10ms/step - loss: 0.0309 - acc: 0.9956
Epoch 6/15
1601/1601 [==============================] - 16s 10ms/step - loss: 0.0297 - acc: 0.9956
Epoch 7/15
1601/1601 [==============================] - 16s 10ms/step - loss: 0.0290 - acc: 0.9956
Epoch 8/15
1601/1601 [==============================] - 16s 10ms/step - loss: 0.0284 - acc: 0.9956
Epoch 9/15
1601/1601 [==============================] - 16s 10ms/step - loss: 0.0281 - acc: 0.9956
Epoch 10/15
1601/1601 [==============================] - 16s 10ms/step - loss: 0.0278 - acc: 0.9956
Epoch 11/

# Evaluate

In [0]:
predictions = model.predict(xTest)

In [0]:
#Distributions of argmins through all the predictions
  
def argminsDistribution(): 
    i = 1
    tab = [0] * 542
    for a in predictions:
      j = 0
      for b in a:
        if b < 1.0 : j += 1

      #print (i, '. ', j, np.argmin(a))
      tab[np.argmin(a)] += 1
      i = i + 1

    i = 0
    for a in tab:
      if a > 0 : print ('position', i, '\targmin count', a)
      i += 1

In [0]:
#failsCount()

In [0]:
#argminsDistribution()

In [0]:
# Evaluate by probability - if the percentage parameter is set to 0.5 than 50% 
# of predictions will be taken into cosideration. When the strongest prediction 
# in given build is 0.9 then predictions from 0.9 to 0.95 will be 
# taken into consideration, if 0.8 than from 0.8 to 0.9

def evaluation2(percentage, predictions, refYsupervisor, verbose=False):
  
  lenght = len(refYsupervisor)
  failPositions = [[] for y in range(lenght)]
  failPredictions = predictions.copy()
  predictionPositions = [[] for y in range(lenght)]
  predictionHits = [0] * lenght
  validPredictionsCount = [0] * lenght

  for i, a in enumerate(refYsupervisor):

    for j, b in enumerate(a):
      if b == 0 : failPositions[i].append(j);
  
  for i, a in enumerate(predictions):
    probabilityLimit = np.amin(a) + ((1 - np.amin(a)) * percentage)

    for j, b in enumerate(a):
      if b <= probabilityLimit: 
        predictionPositions[i].append(j)
        validPredictionsCount[i] += 1
        
  for i, a in enumerate(failPositions):
    count = 0
    
    for b in a:

      for c in predictionPositions[i]:
        if c == b : count += 1

    if len(a) != 0:
      predictionHits[i] = count

  hitsCount = 0
  failsCount = 0
  
  for i in range(lenght):
   # print(predictionHits[i])
    hitsCount += predictionHits[i]
    failsCount += len(failPositions[i])  

  print('Percentage of fails predicted', hitsCount / failsCount * 100, '%',
        '//Average of valid predictions', sum(validPredictionsCount)/len(validPredictionsCount))
  
  if verbose == True:
    for i in range(lenght):
      print(i ,len(failPositions[i]), len(predictionPositions[i]), predictionHits[i], sep='\t')

In [0]:
for i in range(14):
  print(i + 1, ' ', end='')
  evaluation(i, predictions, yTest)

1  additional tests -  1.70 %
2  additional tests -  3.53 %
3  additional tests -  5.35 %
4  additional tests -  6.69 %
5  additional tests -  7.91 %
6  additional tests -  9.37 %
7  additional tests -  10.34 %
8  additional tests -  11.07 %
9  additional tests -  12.17 %
10  additional tests -  13.14 %
11  additional tests -  14.60 %
12  additional tests -  15.21 %
13  additional tests -  16.18 %
14  additional tests -  17.03 %


In [0]:
for i in range(21):
  print(round(0.05 * i, 3), ' ', end='')
  evaluation2(0.05 * i, predictions, yTest)

0.0  Percentage of fails predicted 1.70316301703163 % //Average of valid predictions 1.0
0.05  Percentage of fails predicted 1.70316301703163 % //Average of valid predictions 1.0349127182044888
0.1  Percentage of fails predicted 1.9464720194647203 % //Average of valid predictions 1.0897755610972568
0.15  Percentage of fails predicted 2.068126520681265 % //Average of valid predictions 1.174563591022444
0.2  Percentage of fails predicted 2.5547445255474455 % //Average of valid predictions 1.3690773067331672
0.25  Percentage of fails predicted 3.7712895377128954 % //Average of valid predictions 1.9975062344139651
0.3  Percentage of fails predicted 5.231143552311435 % //Average of valid predictions 2.8728179551122195
0.35  Percentage of fails predicted 5.7177615571776155 % //Average of valid predictions 3.600997506234414
0.4  Percentage of fails predicted 7.177615571776156 % //Average of valid predictions 4.261845386533666
0.45  Percentage of fails predicted 8.880778588807786 % //Average o